In [7]:
from keras.models import model_from_json
import torch
import torchaudio
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import cv2

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded the model from disk")


class Voice_Predict:
    def __init__(self,model):
        self.model=model
    '''This function is used to save spectrogram as images'''
    def create_spectrogram(self,name,array):
        plt.interactive(False)
        fig = plt.figure(figsize=[0.72,0.72])
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)
        S = self.Spectrogram_gen(array) #generate the spectrogram
        librosa.display.specshow(librosa.power_to_db(S, ref=np.max)) #display spectrogram as a image
        filename  = name + '.jpg'
        plt.savefig(filename, dpi=400, bbox_inches='tight',pad_inches=0) #save spectrogram as a image
        plt.close()    
        fig.clf()
        plt.close(fig)
        plt.close('all')
        del name,fig,ax,S

    '''This function is used to generate spectrogrames
    '''
    def Spectrogram_gen(self,array):
        waveform = torch.tensor(array, dtype = torch.float)
        waveform = torch.reshape(waveform, (-1,waveform.shape[0]))
        Spectrogram = torchaudio.transforms.MelSpectrogram()(waveform)[0]
        return (Spectrogram)
    def data_prepare(self,array):
        self.create_spectrogram('predict',array)
        in_img=cv2.resize(cv2.imread('predict.jpg'),(64,64))
        in_img=in_img.astype('float32') / 255
        in_img=np.array([in_img])
        return(in_img)

    def predict(self,array):
        in_img=self.data_prepare(array)
        pred=self.model.predict(in_img)
        predicted_class_indices=np.argmax(pred,axis=1)
        return(predicted_class_indices[0])

Loaded the model from disk


In [8]:
vp=Voice_Predict(model)

In [3]:
import pandas as pd
df_0=pd.read_csv('data sets/Dataset Sound Detection label 0.csv')
df_1=pd.read_csv('data sets/Dataset Sound Detection label 1.csv')

In [9]:
vp.predict(np.array(df_0.iloc[10])[1:-1])

0